Make imports and define functions

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# install required version of rdkit
! pip install rdkit-pypi==2022.3.5

In [3]:
import sys,os
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from rdkit import Chem
from rdkit.Chem import AllChem
import random
from typing import Literal

from itertools import product

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import f_regression, mutual_info_regression, r_regression, SelectKBest
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from math import e
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.svm import SVR

from sklearn.decomposition import PCA

from lightgbm import LGBMRegressor, LGBMClassifier

import json

import sys
sys.path.insert(0, "/content/drive/MyDrive/MLP_final/")
sys.path.insert(0, "/content/drive/MyDrive/MLP_final/ds_packg/")

from ds_packg.data_processing import one_hot_encode, remove_constant_cols, convert_to_canonical_smiles, remove_duplicates, \
                            normalize_columns, scramble_array
from ds_packg.feature_selection import FeatSelector
from ds_packg.model import ModelClass
from ds_packg.method_validation_final import LeaveNoutNx, LeaveNout, CrossVal

import os

def prepare_overall_data(sulfonamide_feats_data_path, chanlam_data_path, structure_cols, df_target_cols, verbose = False):

    #Load and process sulfonamide features
    df_sulfonamide = pd.read_csv(sulfonamide_feats_data_path)
    df_sulfonamide.drop(columns = ['catalyst_name'], inplace = True)
    df_sulfonamide = remove_constant_cols(df_sulfonamide)
    sulfonamide_feats = list(df_sulfonamide.columns[5:])

    #Load and process Chan-Lam data
    df_chan_lam = pd.read_csv(chanlam_data_path, index_col = 0)
    df_chan_lam.dropna(inplace = True)
    df_chan_lam.reset_index(drop= True, inplace = True)
    df_chan_lam.rename(columns={'Sulfonamide':"sulfonamide", 'Boronic Acid':"boronic_acid",
                                'Catalyst':"catalyst", 'Catalyst Name':"catalyst_name", 'Base':"base",
                                'Solvent':"solvent"}, inplace = True)

    df_chan_lam[df_target_cols] = df_chan_lam[df_target_cols]/100
    df_chan_lam = remove_duplicates(df_chan_lam, structure_cols)
    df_chan_lam = df_chan_lam[structure_cols + hand_picked_qm_cols + df_target_cols]

    #Prepare setup and combine both datasets
    df_chan_lam, df_sulfonamide = convert_to_canonical_smiles([df_chan_lam, df_sulfonamide], structure_cols, verbose)
    df_chan_lam_newqm = pd.merge(df_chan_lam, df_sulfonamide, on= structure_cols)

    qm_cols_use = hand_picked_qm_cols+sulfonamide_feats
    df_chan_lam_newqm = df_chan_lam_newqm[structure_cols+qm_cols_use+df_target_cols] #

    #Prepare features for ML/DL
    sulfo_series = df_chan_lam_newqm["sulfonamide"].copy()
    df_chan_lam_newqm, struct_ohe_cols = one_hot_encode(df_chan_lam_newqm, structure_cols)
    df_chan_lam_newqm = normalize_columns(df_chan_lam_newqm, qm_cols_use)
    df_chan_lam_newqm["sulfonamide"] = sulfo_series
    df_chan_lam = df_chan_lam_newqm.sample(frac=1, random_state = rand_state)

    return df_chan_lam, struct_ohe_cols, sulfonamide_feats

def print_performance(score_dict):
    for key, scores in score_dict.items():
        r2_scores = scores['R2']
        mses = scores['RMSE']
        maes = scores['MAE']

        print(key)
        print(f"R2 Score: {np.mean(r2_scores):.5f} ± {np.std(r2_scores):.5f}")
        if np.any(np.array(r2_scores) < 0):
            print(f"Cap Avg R2 Score: {np.mean([v if v >= 0 else 0 for v in r2_scores]):.5f} ± {np.std([v if v >= 0 else 0 for v in r2_scores]):.5f}")
        print(f"MAE: {np.mean(maes):.3%} ± {np.std(maes):.3%}")
        print(f"RMSE: {np.mean(mses):.3%} ± {np.std(mses):.3%}")
        print()

def get_reg_metrics(y_true, y_pred):
    R2 = r2_score(y_true, y_pred)
    MAE = mean_absolute_error(y_true, y_pred)
    RMSE = mean_squared_error(y_true, y_pred, squared=False)
    return R2, MAE, RMSE


def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

def gen_leave_out_list(lo_values: list,
                       lo_n: int=1,
                       random_state: int=42,
                       )->list:
    random.Random(random_state).shuffle(lo_values)
    lo_tests = []
    for i in range(int(len(lo_values)/lo_n)):
        lo_tests.append(lo_values[i*lo_n:(i+1)*lo_n])
    return lo_tests

def read_json(filename: str) -> dict:
    '''
    Read in a json file and return dictionary
    '''
    with open(filename, 'r') as jo:
        json_obj = json.load(jo)

    return json_obj

rand_state = 42

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Read in feature sets identified via feature selection

In [4]:
feats = read_json('/content/drive/MyDrive/MLP_final/Feature_selection.json')

feats.keys()

dict_keys(['Only OHE', 'All Properties', 'DFT', 'NoCoLinear', 'F-regression', 'MutualInfo', 'F-MI_Intersection', 'F-MI_Union', 'RandomForest', 'PCA', 'SHAP'])

In [5]:
df_chan_lam = pd.read_csv('/content/drive/MyDrive/MLP_final/Data/ChanLam_full_dataset_with_features.csv')
df = df_chan_lam.copy()

In [6]:
# generate the list of sulfonamides for LOO

sulfo_series = df['sulfonamide'].copy()

In [7]:
# Define the columns for OHE (structure_cols) and y-output (target_cols)

structure_cols = ['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']
target_cols = ['RAW-MonoYield (%)']
to_drop = ['boronic_acid', 'catalyst', 'catalyst_name', 'base', 'solvent',
           'MonoYield (%)', 'BisYield (%)', 'TotalYield (%)', 'MonoProduct', 'BisProduct',
           'MajorReaction', 'MinorReaction', 'MonoSTD', 'BisSTD', 'TotalSTD', 'Repetitions',
           'CleanLists', 'RAW-MonoSTD', 'RAW-BisYield (%)', 'RAW-BisSTD',
           'RAW-TotalYield (%)', 'RAW-TotalSTD', 'RAW-Repetitions', 'RawLists',
           'RAW-MonoYield-STD Ratio', 'RAW-MonoYield-Range', 'RAW-RangeYield-Fraction']

In [8]:
# one-hot-encode the appropriate columns

df_ssg, struct_ohe_cols = one_hot_encode(df, ['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent'])
df_ssg['sulfonamide'] = sulfo_series
df_ssg[structure_cols] = df[structure_cols].values

validation_sulfonamide_ohe_cols = [f'sulfonamide_{x}' for x in df_ssg[df_ssg['Set']=='Validation']['sulfonamide'].unique()]

for val_ohe in validation_sulfonamide_ohe_cols:
  struct_ohe_cols.remove(val_ohe)
  df_ssg.drop(val_ohe, axis=1, inplace=True)

df_ssg = df_ssg.sample(frac=1, random_state=42)
df_ssg[target_cols] = df_ssg[target_cols]/100

df_ssg.drop(to_drop, axis=1, inplace=True)

Modeling

In [9]:
# set hyperparameters for MLP

hyperparams_single = {
    'hidden_size': 375,
    'drop_out_rate': 0.55,
    'step_schedule': 80,
    'epochs': 200,
    'batch_size': 125,
    'output_size': 1,
    'l1_reg' : 0,
    'l2_reg' : 0
}

hidden_size, drop_out_rate, step_schedule, num_epochs, batch_size, output_size, l1_reg, l2_reg = hyperparams_single.values()

30 random 80/20 splits

In [13]:
# MLP metric

cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']
df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

#Initialize model class
input_size = len(df_.columns) - len(target_cols) - 1
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize CV
CV = CrossVal(df_.drop(columns=['sulfonamide']), MC, batch_size, rand_state=rand_state, n_cv=30)

#Run training loop with CV
CV.execute_loop(target_cols, output_size)
CV.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=CV.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)

to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_random.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

Cross-validation fold: 1/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:353: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
  3%|▎         | 1/30 [00:48<23:16, 48.15s/it]

Fold 1 R2 Score: 0.80135, MSE: 0.09389, MAE: 0.06591
Cross-validation fold: 2/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  7%|▋         | 2/30 [01:36<22:27, 48.12s/it]

...

Fold 28 R2 Score: 0.80058, MSE: 0.09674, MAE: 0.06857
Cross-validation fold: 29/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
 97%|█████████▋| 29/30 [22:56<00:47, 47.14s/it]

Fold 29 R2 Score: 0.80497, MSE: 0.09056, MAE: 0.06420
Cross-validation fold: 30/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 30/30 [23:43<00:00, 47.43s/it]


Fold 30 R2 Score: 0.78768, MSE: 0.09728, MAE: 0.06784
R2 Score: 0.79723 ± 0.01369
Cap Avg R2 Score: 0.79723 ± 0.01369
MAE: 6.608% ± 0.274%
RMSE: 9.381% ± 0.405%


In [14]:
# 30 random 80/20 splits with other models: chemically informed; scale each split individually instead of train / test at once

models = [LGBMRegressor(
                 learning_rate=0.01, # 0.01
                 n_estimators=1024, # 1024
                 reg_alpha=0.01, # 0.01
                 reg_lambda=0.01, # 0.01
                 verbose=-1,
                 n_jobs=-1,
                 random_state=rand_state),

          RandomForestRegressor(
                 verbose=False,
                 n_jobs=-1,
                 n_estimators=1024,
                 min_samples_split=2,
                 random_state=rand_state),

          SVR(
                 verbose=False,
                 kernel='rbf',
                 C=0.25,
                 epsilon=0.1,
                 gamma='scale')]

model_names = ['LGBM', 'RF', 'SVM']

cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']
df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

results = {}

# Initialize lists
results = {}
predictions_dict = {}

# Initialize lists
for name, model in tqdm(zip(model_names, models)):
    results[name] = {}
    predictions_dict[name] = {'Index': [], 'True': [], 'Predicted': []}

    r2_list = []
    mae_list = []
    rmse_list = []
    iter_list = []

    for i in tqdm(range(0, 30)):
        iter_list.append(i)

        # Split data
        df_train, df_test = train_test_split(df_, random_state=i, train_size=0.8)

        # Save test indices
        test_indices = df_test.index.tolist()

        X_train, X_test = df_train.drop(columns=['RAW-MonoYield (%)', 'sulfonamide']).values, df_test.drop(columns=['RAW-MonoYield (%)', 'sulfonamide']).values
        y_train, y_test = np.asarray(df_train['RAW-MonoYield (%)'].tolist(), dtype=np.float64), np.asarray(df_test['RAW-MonoYield (%)'].tolist(), dtype=np.float64)

        ## Scale due to having QM descriptors
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Fit the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Store test indices, true and predicted values
        predictions_dict[name]['Index'].extend(test_indices)
        predictions_dict[name]['True'].extend(y_test)
        predictions_dict[name]['Predicted'].extend(y_pred)

        # Calculate metrics
        R2, MAE, RMSE = get_reg_metrics(y_true=y_test, y_pred=y_pred)

        r2_list.append(R2)
        mae_list.append(MAE)
        rmse_list.append(RMSE)

        r2_array = np.asarray(r2_list, dtype=np.float64)
        mae_array = np.asarray(mae_list, dtype=np.float64)
        rmse_array = np.asarray(rmse_list, dtype=np.float64)

    results[name] = {'CV_Iter': iter_list, 'R2': r2_array, 'MAE': mae_array, 'RMSE': rmse_array}

# Save true vs. predicted values as DataFrames
for name in model_names:
    df_pred = pd.DataFrame(predictions_dict[name])
    mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=df_pred.set_index('Index'), left_index=True, right_index=True)
    to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
    to_save.to_csv(f'/content/drive/MyDrive/MLP_final/{name}_preds_random.csv')

0it [00:00, ?it/s]
  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  3%|▎         | 1/30 [00:01<00:32,  1.13s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋         | 2/30 [00:02<00:30,  1.11s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 10%|█         | 3/30 [00:03<00:29,  1.10s/it]/usr/local/lib/python3

MLP baseline models

In [15]:
# X-scrambled baseline model
cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']
df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

#Initialize model class
input_size = len(df_.columns) - len(target_cols) - 1
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize CV
CV = CrossVal(df_.drop(columns=['sulfonamide']), MC, batch_size, rand_state=rand_state, n_cv=30, scramble_X=True)

#Run training loop with CV
CV.execute_loop(target_cols, output_size)
CV.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=CV.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_Xscrambled.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

Cross-validation fold: 1/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:353: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
  3%|▎         | 1/30 [00:47<23:08, 47.89s/it]

Fold 1 R2 Score: -0.33436, MSE: 0.24334, MAE: 0.18363
Cross-validation fold: 2/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  7%|▋         | 2/30 [01:34<22:00, 47.16s/it]

...

Fold 28 R2 Score: -0.34682, MSE: 0.25140, MAE: 0.19292
Cross-validation fold: 29/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
 97%|█████████▋| 29/30 [22:20<00:45, 45.73s/it]

Fold 29 R2 Score: -0.37892, MSE: 0.24080, MAE: 0.18587
Cross-validation fold: 30/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 30/30 [23:07<00:00, 46.27s/it]


Fold 30 R2 Score: -0.25314, MSE: 0.23634, MAE: 0.17857
R2 Score: -0.36779 ± 0.05823
Cap Avg R2 Score: 0.00000 ± 0.00000
MAE: 18.674% ± 0.636%
RMSE: 24.376% ± 0.904%


In [16]:
# y-scrambled baseline model
cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']
df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

#Initialize model class
input_size = len(df_.columns) - len(target_cols) - 1
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize CV
CV = CrossVal(df_.drop(columns=['sulfonamide']), MC, batch_size, rand_state=rand_state, n_cv=30, scramble_y=True)

#Run training loop with CV
CV.execute_loop(target_cols, output_size)
CV.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=CV.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_yscrambled.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

Cross-validation fold: 1/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:353: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
  3%|▎         | 1/30 [00:48<23:34, 48.79s/it]

Fold 1 R2 Score: -0.35324, MSE: 0.24506, MAE: 0.18915
Cross-validation fold: 2/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  7%|▋         | 2/30 [01:34<21:57, 47.04s/it]

...

Fold 28 R2 Score: -0.32651, MSE: 0.24950, MAE: 0.18860
Cross-validation fold: 29/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
 97%|█████████▋| 29/30 [22:29<00:46, 46.34s/it]

Fold 29 R2 Score: -0.33855, MSE: 0.23725, MAE: 0.18099
Cross-validation fold: 30/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 30/30 [23:16<00:00, 46.55s/it]


Fold 30 R2 Score: -0.26093, MSE: 0.23708, MAE: 0.18378
R2 Score: -0.34668 ± 0.06109
Cap Avg R2 Score: 0.00000 ± 0.00000
MAE: 18.537% ± 0.573%
RMSE: 24.184% ± 0.849%


In [17]:
# ohe baseline model
cols_to_use = struct_ohe_cols + target_cols + ['Set', 'sulfonamide']
df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

#Initialize model class
input_size = len(df_.columns) - len(target_cols) - 1
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize CV
CV = CrossVal(df_.drop(columns=['sulfonamide']), MC, batch_size, rand_state=rand_state, n_cv=30)

#Run training loop with CV
CV.execute_loop(target_cols, output_size)
CV.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=CV.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_OHE.csv')

  0%|          | 0/30 [00:00<?, ?it/s]

Cross-validation fold: 1/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:353: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
  3%|▎         | 1/30 [00:45<22:10, 45.86s/it]

Fold 1 R2 Score: 0.79203, MSE: 0.09607, MAE: 0.06903
Cross-validation fold: 2/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  7%|▋         | 2/30 [01:32<21:31, 46.14s/it]

...

Fold 28 R2 Score: 0.79507, MSE: 0.09807, MAE: 0.07016
Cross-validation fold: 29/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
 97%|█████████▋| 29/30 [22:00<00:45, 45.43s/it]

Fold 29 R2 Score: 0.78663, MSE: 0.09472, MAE: 0.06681
Cross-validation fold: 30/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 30/30 [22:45<00:00, 45.50s/it]


Fold 30 R2 Score: 0.75572, MSE: 0.10435, MAE: 0.07234
R2 Score: 0.78399 ± 0.01510
Cap Avg R2 Score: 0.78399 ± 0.01510
MAE: 6.837% ± 0.287%
RMSE: 9.682% ± 0.436%


Leave-one-out cross validation

In [18]:
# chemically informed model for LOOCV

cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']

df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

output_size = 1
input_size = len(df_.columns) - len(target_cols) - 1

feat_results = pd.DataFrame(columns=['LO-SMILES','R2','RMSE','MAE']+['FeatureSet'])

MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize Leave-N-Out Validation
LNO = LeaveNout(df_, "sulfonamide", MC, batch_size, rand_state=rand_state)

#Run training loop with Leave-N-Out Validation
LNO.execute_loop(target_cols, output_size)

feat_mlp = pd.DataFrame(data=np.stack([flatten_extend(LNO.test_list),LNO.r2_scores,LNO.mses,LNO.maes],axis=1),columns=['LO-SMILES','R2','RMSE','MAE'])

feat_results = pd.concat([feat_results,feat_mlp],ignore_index=True)

LNO.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=LNO.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_loocv.csv')

  0%|          | 0/22 [00:00<?, ?it/s]/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:235: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  5%|▍         | 1/22 [00:54<19:10, 54.77s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use

R2 Score: 0.25859 ± 0.68745
Cap Avg R2 Score: 0.42141 ± 0.28343
MAE: 11.116% ± 5.631%
RMSE: 14.314% ± 6.745%


In [12]:
# OHE baseline model for LOOCV prediction

cols_to_use = struct_ohe_cols + target_cols + ['Set', 'sulfonamide']

df_ = df_ssg[cols_to_use].copy()
df_ = df_[df_['Set']=='Train'].copy().drop('Set', axis=1)

output_size = 1
input_size = len(df_.columns) - len(target_cols) - 1

feat_results = pd.DataFrame(columns=['LO-SMILES','R2','RMSE','MAE']+['FeatureSet'])

MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state)

#Initialize Leave-N-Out Validation
LNO = LeaveNout(df_, "sulfonamide", MC, batch_size, rand_state=rand_state)

#Run training loop with Leave-N-Out Validation
LNO.execute_loop(target_cols, output_size)

feat_mlp = pd.DataFrame(data=np.stack([flatten_extend(LNO.test_list),LNO.r2_scores,LNO.mses,LNO.maes],axis=1),columns=['LO-SMILES','R2','RMSE','MAE'])

feat_results = pd.concat([feat_results,feat_mlp],ignore_index=True)

LNO.print_performance()

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=LNO.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_ohe_loocv.csv')

  0%|          | 0/22 [00:00<?, ?it/s]/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:235: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
  5%|▍         | 1/22 [01:01<21:21, 61.01s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use

R2 Score: 0.12281 ± 0.84913
Cap Avg R2 Score: 0.38132 ± 0.26369
MAE: 12.025% ± 6.142%
RMSE: 15.309% ± 7.053%


External validation

In [19]:
# external validation: train using train and test using validation

cols_to_use = struct_ohe_cols + feats['SHAP'] + target_cols + ['Set', 'sulfonamide', 'SO2N-dG_deprotonation_gas', 'C_Hirschfeld', 'N_Hirschfeld']
df_ = df_ssg[cols_to_use].copy()

#Initialize model class
metrics_ = []

hidden_size, drop_out_rate, step_schedule, num_epochs, batch_size, output_size, l1_reg, l2_reg = hyperparams_single.values()

r2_list = []
rmse_list = []
mae_list = []

output_size = 1

# separate train and test dfs
df_train = df_[df_['Set']=='Train'].reset_index(drop=True).copy()
df_train = df_train.sample(frac=1, random_state=rand_state).reset_index(drop=False).rename(columns={'index': 'original_index'})
df_val = df_[df_['Set']=='Validation'].reset_index(drop=False).copy().rename(columns={'index': 'original_index'})

df_external = pd.concat([df_train, df_val], ignore_index=True).drop('sulfonamide', axis=1)

input_size = len(df_external.columns) - len(target_cols) - 2

# initialize model
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state=rand_state)

# initialize LNO (where n=all external molecules as determined by "set")
LNO = LeaveNout(df_.drop('sulfonamide', axis=1), 'Set', MC, batch_size, rand_state=rand_state)

# run training loop
LNO.execute_loop(target_cols, output_size)

# performance metrics
val_idx = flatten_extend(LNO.test_list).index('Validation')
r2_val, rmse_val, mae_val = LNO.r2_scores[val_idx], LNO.mses[val_idx], LNO.maes[val_idx]

r2_list.append(r2_val)
rmse_list.append(rmse_val)
mae_list.append(mae_val)

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=LNO.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_external.csv')

  0%|          | 0/2 [00:00<?, ?it/s]/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:235: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 2/2 [00:57<00:00, 28.72s/it]


In [20]:
# OHE baseline model for external test prediction

cols_to_use = struct_ohe_cols + target_cols + ['Set', 'sulfonamide']
df_ = df_ssg[cols_to_use].copy()

#Initialize model class
metrics_ = []

hidden_size, drop_out_rate, step_schedule, num_epochs, batch_size, output_size, l1_reg, l2_reg = hyperparams_single.values()

r2_list = []
rmse_list = []
mae_list = []

output_size = 1

# separate train and test dfs
df_train = df_[df_['Set']=='Train'].reset_index(drop=True).copy()
df_train = df_train.sample(frac=1, random_state=rand_state).reset_index(drop=False).rename(columns={'index': 'original_index'})
df_val = df_[df_['Set']=='Validation'].reset_index(drop=False).copy().rename(columns={'index': 'original_index'})

df_external = pd.concat([df_train, df_val], ignore_index=True).drop('sulfonamide', axis=1)

input_size = len(df_external.columns) - len(target_cols) - 2

# initialize model
MC = ModelClass("MLP", input_size, hidden_size, output_size, drop_out_rate, step_schedule, num_epochs, l1_reg, l2_reg, rand_state=rand_state)

# initialize LNO (where n=all external molecules as determined by "set")
LNO = LeaveNout(df_.drop('sulfonamide', axis=1), 'Set', MC, batch_size, rand_state=rand_state)

# run training loop
LNO.execute_loop(target_cols, output_size)

# performance metrics
val_idx = flatten_extend(LNO.test_list).index('Validation')
r2_val, rmse_val, mae_val = LNO.r2_scores[val_idx], LNO.mses[val_idx], LNO.maes[val_idx]

r2_list.append(r2_val)
rmse_list.append(rmse_val)
mae_list.append(mae_val)

mlp_preds = pd.merge(left=df['RAW-MonoYield (%)'], right=LNO.predictions_df.set_index('Index'), left_index=True, right_index=True)
to_save = pd.merge(left=mlp_preds, right=df[['sulfonamide', 'boronic_acid', 'catalyst', 'base', 'solvent']], left_index=True, right_index=True)
to_save.to_csv('/content/drive/MyDrive/MLP_final/mlp_preds_ohe_external.csv')

  0%|          | 0/2 [00:00<?, ?it/s]/content/drive/MyDrive/MLP_final/ds_packg/method_validation_final.py:235: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictions_df = pd.concat([self.predictions_df, fold_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 2/2 [00:57<00:00, 28.59s/it]
